In [ ]:
'''
ПЛАН РАБОТ:
- заполнить пробелы в 1-10 созывах (спорт-9, партнеры-8, стипендии-7)
'''

In [1]:
import requests
from bs4 import Tag, NavigableString, BeautifulSoup
import re
from datetime import datetime, date, time

import pandas as pd


In [165]:
ssmm_candidates_core = pd.read_csv('../library: main/ssmm_candidates_core.csv', sep='\t', encoding='utf-8')
ssmm_candidates_core = ssmm_candidates_core[['name','ssmm_id']]
# ssmm_candidates_core.head()

name_list = list(ssmm_candidates_core['name'])
# print(name_list)

def short_to_full(short_name):
    fff = [name for name in name_list if short_name in name]
    if len(fff) > 0:
        return fff[0]
    else:
        return short_name

In [64]:
# 

# req_announcement = requests.get('https://vk.com//pages?oid=-33327655&p=%D0%A1%D0%BE%D1%81%D1%82%D0%B0%D0%B2_10_%D0%BC%D0%B0%D1%80%D1%82_21')
# html_doc = req_announcement.text
# print(html_doc)



In [65]:
# ИЗВЛЕКАЕМ СПИСОК ССЫЛОК СО СТРАНИЦЫ В МЕНЮ

req_link = requests.get('https://vk.com/pages?oid=-33327655&p=%D0%A1%D0%9E%D0%A1%D0%A2%D0%90%D0%92_%D0%A1%D1%82%D1%83%D0%B4%D1%81%D0%BE%D0%B2%D0%B5%D1%82%D0%B0_%D0%BC%D0%B5%D1%85%D0%BC%D0%B0%D1%82%D0%B0')

html_doc = req_link.text
soup = BeautifulSoup(html_doc, 'html.parser')

strings = soup.find_all('span', attrs={'class': 'l'})

menu_dict = {}

for s in strings:
    key = [line for line in s.stripped_strings][0]
    menu_dict[key] = 'https://vk.com/' + s.find('a', attrs={'href': True})['href']

# menu_dict


In [66]:
# <table cellpadding="0" cellspacing="0" class="wk_table">

def menu_page_to_dict(vk_menu_link):
    req_link = requests.get(vk_menu_link)
    html_doc = req_link.text
    soup = BeautifulSoup(html_doc, 'html.parser')
    strings = soup.find_all('table', attrs={'class': 'wk_table'})
    return strings

# menu_page_to_dict(menu_dict['4 состав — ноябрь 2014'])

In [47]:
# список ссылок на страницы в меню по составу каждого созыва

# import copy

menu_dict_upd = {}

for key in menu_dict.keys():
    if "'" not in key:
        menu_dict_upd[key] = menu_dict[key]

# menu_dict_upd = copy.deepcopy(menu_dict)
del menu_dict_upd['0 («нелегитимный») состав - декабрь 2011']
del menu_dict_upd['10 состав — март 2021 (после пятого собрания Студсовета)']

menu_dict_upd

{'10 состав — ноябрь 2020': 'https://vk.com//pages?oid=-33327655&p=%D0%A1%D0%BE%D1%81%D1%82%D0%B0%D0%B2_10_%D0%BD%D0%BE%D1%8F%D0%B1%D1%80%D1%8C_20',
 '9 состав — ноябрь 2019': 'https://vk.com//pages?oid=-33327655&p=%D0%A1%D0%BE%D1%81%D1%82%D0%B0%D0%B2_%D0%A1%D1%82%D1%83%D0%B4%D1%81%D0%BE%D0%B2%D0%B5%D1%82%D0%B0_%D0%BC%D0%B5%D1%85%D0%BC%D0%B0%D1%82%D0%B0%2C_2019-11',
 '8 состав — ноябрь 2018': 'https://vk.com//pages?oid=-33327655&p=%D0%A1%D0%BE%D1%81%D1%82%D0%B0%D0%B2_%D0%A1%D1%82%D1%83%D0%B4%D1%81%D0%BE%D0%B2%D0%B5%D1%82%D0%B0_%D0%BC%D0%B5%D1%85%D0%BC%D0%B0%D1%82%D0%B0%2C_2018-11',
 '7 состав — ноябрь 2017': 'https://vk.com//pages?oid=-33327655&p=%D0%A1%D0%BE%D1%81%D1%82%D0%B0%D0%B2_%D0%A1%D1%82%D1%83%D0%B4%D1%81%D0%BE%D0%B2%D0%B5%D1%82%D0%B0_%D0%BC%D0%B5%D1%85%D0%BC%D0%B0%D1%82%D0%B0%2C_2017-10',
 '6 состав — ноябрь 2016': 'https://vk.com//pages?oid=-33327655&p=%D0%A1%D0%BE%D1%81%D1%82%D0%B0%D0%B2_%D0%A1%D1%82%D1%83%D0%B4%D1%81%D0%BE%D0%B2%D0%B5%D1%82%D0%B0_%D0%BC%D0%B5%D1%85%D0%BC%D0

In [89]:
# wrong : correct

def replace_position(position):
    replace_position_names = {
        'Ответственный секретарь': 'Секретарь',
        'Представители в Студсовет МГУ': 'Представитель в Студсовете МГУ',
        'Представители в Студсовете МГУ': 'Представитель в Студсовете МГУ',
        'Представитель в Учёном совете': 'Представитель в Учёном совете мехмата',
        'Представитель в Учёный совет мехмата': 'Представитель в Учёном совете мехмата',
        'Представитель в Учёном совете мехмата МГУ': 'Представитель в Учёном совете мехмата',
        'Заместители представителей': 'Заместитель представителей в Студсовете МГУ',
        'Бытовые вопросы (ГЗ А, Б, Г, Е, ДСЛ, ФДС, экология)': 'Бытовые вопросы',
        'Бытовые вопросы (сектора «А» и «Б», ФДС, экология)': 'Бытовые вопросы'
    }
    if position in replace_position_names.keys():
        return replace_position_names[position]
    else:
        return position

# РУКОВОДСТВО

In [166]:

from functools import reduce

management_dict = {}

for key in menu_dict_upd.keys():
    aaa = {}
    table = menu_page_to_dict(menu_dict_upd[key])
    for s in table[0]:
        lul = [line for line in s.stripped_strings]
        if replace_position(lul[0]) == 'Представитель в Студсовете МГУ':
            list_of = [nn.strip() for nn in reduce(lambda x,y: x+y, [name.split(',') for name in lul[1:] if len(name) > 2])]
            aaa['Первый представитель в Студсовете МГУ'] = short_to_full(list_of[0][:list_of[0].index('.')])
            aaa['Второй представитель в Студсовете МГУ'] = short_to_full(list_of[1][:list_of[1].index('.')])
        elif replace_position(lul[0]) == 'Заместитель представителей в Студсовете МГУ':
            list_of = [nn.strip() for nn in reduce(lambda x,y: x+y, [name.split(',') for name in lul[1:] if len(name) > 2])]
            aaa['Первый заместитель представителей в Студсовете МГУ'] = short_to_full(list_of[0][:list_of[0].index('.')])
            if len(list_of) > 1:
                aaa['Второй заместитель представителей в Студсовете МГУ'] = short_to_full(list_of[1][:list_of[1].index('.')])
        elif '.' in lul[1]:
            aaa[replace_position(lul[0])] = short_to_full(lul[1][:lul[1].index('.')])
        else:
            aaa[replace_position(lul[0])] = short_to_full(lul[1])
#         if replace_position(lul[0]) in ['Представитель в Студсовете МГУ','Заместитель представителей в Студсовете МГУ']:
#             aaa[replace_position(lul[0])] = [nn.strip() for nn in reduce(lambda x,y: x+y, [name.split(',') for name in lul[1:] if len(name) > 2])]
#         else:
#             aaa[replace_position(lul[0])] = lul[1]
    del aaa['Должность']
    management_dict[int(key[:key.index(' ')])] = aaa

management_dict



{10: {'Председатель': 'Малиновский Георгий Андреевич',
  'Заместитель председателя': 'Шишков Александр Александрович',
  'Секретарь': 'Мельникова Дарья Владимировна',
  'Первый представитель в Студсовете МГУ': 'Сулейкин Аллан Олегович',
  'Второй представитель в Студсовете МГУ': 'Третьяков Пётр Максимович',
  'Первый заместитель представителей в Студсовете МГУ': 'Кухарчук Иван Андреевич',
  'Второй заместитель представителей в Студсовете МГУ': 'Скурихин Дмитрий Андреевич',
  'Представитель в Учёном совете мехмата': 'Клюев Андрей Игоревич'},
 9: {'Председатель': 'Клюев Андрей Игоревич',
  'Заместитель председателя': 'Фенелонов Даниил Игоревич',
  'Секретарь': 'Попова Анна Романовна',
  'Первый представитель в Студсовете МГУ': 'Клюев Андрей Игоревич',
  'Второй представитель в Студсовете МГУ': 'Кухарчук Иван Андреевич',
  'Первый заместитель представителей в Студсовете МГУ': 'Малахов Николай Сергеевич',
  'Представитель в Учёном совете мехмата': 'Третьяков Пётр Максимович'},
 8: {'Предсе

In [167]:
# добавить 11 созыв:
# https://vk.com/wall-33327655_10064

def text_by_link(link):
    req_link = requests.get(link)
    html_doc = req_link.text
    soup = BeautifulSoup(html_doc, 'html.parser')

    wi_body = soup.find('div', attrs={'class': 'wi_body'})
    pi_text = wi_body.find('div', attrs={'class': 'pi_text'})

    conv_11_dict = {}
    flg = 0

    for s in pi_text.stripped_strings:
        if '—' in s:
            position = replace_position(s[:s.index('—')].strip())
            flg = 1
        elif '–' in s:
            position = replace_position(s[:s.index('–')].strip())
            flg = 1
        elif flg == 1:
            name = s.split()
            conv_11_dict[position] = short_to_full(name[1]+' '+name[0][0])
            flg = 0
    return conv_11_dict

management_11_dict = text_by_link('https://vk.com/wall-33327655_10064')
management_11_dict

{'Председатель': 'Малиновский Георгий Андреевич',
 'Заместитель председателя': 'Дехнич Кирилл Андреевич',
 'Секретарь': 'Мельникова Дарья Владимировна',
 'Первый представитель в Студсовете МГУ': 'Дехнич Кирилл Андреевич',
 'Второй представитель в Студсовете МГУ': 'Третьяков Пётр Максимович',
 'Представитель в Учёном совете мехмата': 'Чуб Вячеслав Валериевич'}

In [168]:

management_dict[11] = {}
for key in management_11_dict.keys():
    management_dict[11][key] = management_11_dict[key]


In [169]:
management_df =  pd.DataFrame(management_dict.items(), columns=['convocation_no', 'params'])
management_df_2 = pd.concat([management_df.drop(['params'], axis=1), management_df['params'].apply(pd.Series)], axis=1)
# management_df_2[['Первый представитель в Студсовете МГУ','Второй представитель в Студсовете МГУ']] = pd.DataFrame(management_df_2['Представитель в Студсовете МГУ'].tolist(), index= management_df_2.index)
# management_df_2 = management_df_2.drop(['Представитель в Студсовете МГУ'], axis=1)
management_df_2 = management_df_2.sort_values(['convocation_no'],
              ascending = [True]).reset_index().drop(['index'], axis=1)

management_df_2


,convocation_no,Председатель,Заместитель председателя,Секретарь,Первый представитель в Студсовете МГУ,Второй представитель в Студсовете МГУ,Первый заместитель представителей в Студсовете МГУ,Второй заместитель представителей в Студсовете МГУ,Представитель в Учёном совете мехмата
0,1,Котляров Никита Владимирович,Асташов Дмитрий Сергеевич,Тропин Александр Михайлович,Котляров Никита Владимирович,Савушкин Никита Максимович,Тропин Александр Михайлович,Владыкина Вероника Евгеньевна,Владыкина Вероника Евгеньевна
1,2,Сапунов Кирилл Вячеславович,Попов Леонид Андреевич,Тропин Александр Михайлович,Сапунов Кирилл Вячеславович,Тропин Александр Михайлович,Осин Руслан Владимирович,Ватутин Кирилл Александрович,Владыкина Вероника Евгеньевна
2,3,Матвеев Алексей Викторович,Быстрицкая Василина Васильевна,Дяченко Мария Игоревна,Матвеев Алексей Викторович,Сапунов Кирилл Вячеславович,Осин Руслан Владимирович,Ватутин Кирилл Александрович,NaN
3,4,Глунчадзе Ираклий Вахтангович,Быстрицкая Василина Васильевна,Дяченко Мария Игоревна,Глунчадзе Ираклий Вахтангович,Матвеев Алексей Викторович,Митькина Мария Александровна,Шведов Константин Анатольевич,NaN
4,5,Бекбулатов Рамзан Ломалиевич,Денисов Илья Олегович,Савин Павел Андреевич,Бекбулатов Рамзан Ломалиевич,Шведов Константин Анатольевич,Морозов Сергей Сергеевич,Суркова Анастасия Юрьевна,NaN
5,6,Денисов Илья Олегович,Бекбулатов Рамзан Ломалиевич,Чернова Анна Александровна,Денисов Илья Олегович,Бекбулатов Рамзан Ломалиевич,Семенюк Павел Игоревич,Корж Даниил Сергеевич,NaN
6,7,Тонконогов Александр Владиславович,Сенотов Валерий Дмитриевич,Кудасов Матвей Джамалханович,Тонконогов Александр Владиславович,Денисов Илья Олегович,Сенотов Валерий Дмитриевич,Корж Даниил Сергеевич,NaN
7,8,Сенотов Валерий Дмитриевич,Хахова Мария Алексеевна,Малиновский Георгий Андреевич,Сенотов Валерий Дмитриевич,Денисов Илья Олегович,Тонконогов Александр Владиславович,Малахов Николай Сергеевич,NaN
8,9,Клюев Андрей Игоревич,Фенелонов Даниил Игоревич,Попова Анна Романовна,Клюев Андрей Игоревич,Кухарчук Иван Андреевич,Малахов Николай Сергеевич,NaN,Третьяков Пётр Максимович
9,10,Малиновский Георгий Андреевич,Шишков Александр Александрович,Мельникова Дарья Владимировна,Сулейкин Аллан Олегович,Третьяков Пётр Максимович,Кухарчук Иван Андреевич,Скурихин Дмитрий Андреевич,Клюев Андрей Игоревич


In [222]:
management_df_3 = management_df_2.melt(id_vars=["convocation_no"]).rename(columns={'variable':'position', 'value':'name'})
management_df_3 = management_df_3[management_df_3['name'].notna()].reset_index().drop(['index'], axis=1)
management_df_3



,convocation_no,position,name
0,1,Председатель,Котляров Никита Владимирович
1,2,Председатель,Сапунов Кирилл Вячеславович
2,3,Председатель,Матвеев Алексей Викторович
3,4,Председатель,Глунчадзе Ираклий Вахтангович
4,5,Председатель,Бекбулатов Рамзан Ломалиевич
...,...,...,...
74,1,Представитель в Учёном совете мехмата,Владыкина Вероника Евгеньевна
75,2,Представитель в Учёном совете мехмата,Владыкина Вероника Евгеньевна
76,9,Представитель в Учёном совете мехмата,Третьяков Пётр Максимович
77,10,Представитель в Учёном совете мехмата,Клюев Андрей Игоревич


In [228]:
management_df_3.to_csv('ssmm_management_1_11.csv', sep='\t', encoding='utf-8', index=False)
management_df_3.to_csv('../library: main/ssmm_management_1_11.csv', sep='\t', encoding='utf-8', index=False)


In [209]:
def pos_type(position):
    if position in ['Председатель','Заместитель председателя','Секретарь']:
        return 'Руководящая должность'
    elif 'представител' in str.lower(position):
        return 'Представительская должность'

def is_vice(position):
    if 'заместитель' in str.lower(position):
        return 1
    else:
        return 0

def is_SSMGU_member(position):
    if 'представитель в Студсовете МГУ' in position:
        return 1
    else:
        return 0

def is_management(position):
    if 'заместитель представителей' in position:
        return 0
    else:
        return 1
    
dict_management = management_df_3.groupby(['position']).agg({'convocation_no':'min'}).reset_index().rename(columns={'convocation_no':'first_convocation_no'})
dict_management['position_type'] = dict_management['position'].apply(lambda x: pos_type(x) )
dict_management['management_flg'] = dict_management['position'].apply(lambda x: is_management(x) )
dict_management['MSU_council_member_flg'] = dict_management['position'].apply(lambda x: is_SSMGU_member(x) )
dict_management['vice_flg'] = dict_management['position'].apply(lambda x: is_vice(x) )
dict_management = dict_management.sort_values(['position_type', 'MSU_council_member_flg'],
              ascending = [False, False]).reset_index().drop(['index'], axis=1)
dict_management


,position,first_convocation_no,position_type,management_flg,MSU_council_member_flg,vice_flg
0,Заместитель председателя,1,Руководящая должность,1,0,1
1,Председатель,1,Руководящая должность,1,0,0
2,Секретарь,1,Руководящая должность,1,0,0
3,Второй представитель в Студсовете МГУ,1,Представительская должность,1,1,0
4,Первый представитель в Студсовете МГУ,1,Представительская должность,1,1,0
5,Второй заместитель представителей в Студсовете...,1,Представительская должность,0,0,1
6,Первый заместитель представителей в Студсовете...,1,Представительская должность,0,0,1
7,Представитель в Учёном совете мехмата,1,Представительская должность,1,0,0


In [227]:
dict_management.to_csv('ssmm_dict_management.csv', sep='\t', encoding='utf-8', index=False)
dict_management.to_csv('../dictionaries/ssmm_dict_management.csv', sep='\t', encoding='utf-8', index=False)


# КООРДИНАТОРЫ И РАБОЧИЕ ГРУППЫ

In [170]:

from functools import reduce

coordinators_dict = {}

for key in menu_dict_upd.keys():
    aaa = {}
    table = menu_page_to_dict(menu_dict_upd[key])
    for s in table[1]:
        lul = [line for line in s.stripped_strings]
        if 'рабочей группы' not in lul[0]:
            if len(lul) > 1:
                if '.' in lul[1]:
                    aaa[replace_position(lul[0])] = short_to_full(lul[1][:lul[1].index('.')])
                else:
                    aaa[replace_position(lul[0])] = short_to_full(lul[1])
            else:
                aaa[replace_position(lul[0])] = '-'
    coordinators_dict[int(key[:key.index(' ')])] = aaa

coordinators_dict


{10: {'Бытовые вопросы': 'Дехнич Кирилл Андреевич',
  'Образование': 'Кухарчук Иван Андреевич',
  'Оповещение': 'Третьяков Пётр Максимович',
  'Спорт': 'Евдокимова Софья Александровна',
  'Праздники': 'Манахова Анастасия Николаевна',
  'Связи': 'Чуб Вячеслав Валериевич',
  'Распределение стипендий': 'Шишков Александр Александрович'},
 9: {'Бытовые вопросы': 'Фенелонов Даниил Игоревич',
  'Образование': 'Саметов Максим Алексеевич',
  'Оповещение': 'Малиновский Георгий Андреевич',
  'Спорт': 'Определится на собрании рабочей группы',
  'Праздники': 'Евдокимова Софья Александровна',
  'Распределение стипендий': 'Кухарчук Иван Андреевич'},
 8: {'Бытовые вопросы': 'Корж Даниил Сергеевич',
  'Образование': 'Промыслов Платон Валерьевич',
  'Оповещение': 'Краснова Анастасия Александровна',
  'Спорт': 'Малахов Николай Сергеевич',
  'Праздники': 'Вихорева Надежда Дмитриевна',
  'Работа с партнёрами': 'Определится на собрании рабочей группы',
  'Распределение стипендий': 'Кухарчук Иван Андреевич'}

In [171]:
# добавить 11 созыв:
# https://vk.com/wall-33327655_10092

coordinators_11_dict = text_by_link('https://vk.com/wall-33327655_10092')
coordinators_11_dict


{'Бытовые вопросы': 'Приходко Макар Александрович',
 'Образование': 'Лоншаков Никита Артемович',
 'Оповещение': 'Алёшечкина Екатерина Сергеевна',
 'Праздники': 'Манахова Анастасия Николаевна',
 'Спорт': 'Шевченко Екатерина Романовна'}

In [184]:
# заполнить пробелы:

coordinators_dict[4]['Распределение стипендий'] = short_to_full('Матвеев Д')
coordinators_dict[5]['Распределение стипендий'] = short_to_full('Подкользина Е')
coordinators_dict[6]['Распределение стипендий'] = short_to_full('Глунчадзе И')
coordinators_dict[7]['Распределение стипендий'] = None
coordinators_dict[7]['Образование'] = short_to_full('Воробьёва Н') # Е/Ё
coordinators_dict[8]['Работа с партнёрами'] = None
coordinators_dict[9]['Спорт'] = None

coordinators_dict[11] = {}
for key in coordinators_11_dict.keys():
    coordinators_dict[11][key] = coordinators_11_dict[key]


In [185]:
coordinators_df =  pd.DataFrame(coordinators_dict.items(), columns=['convocation_no', 'params'])
coordinators_df_2 = pd.concat([coordinators_df.drop(['params'], axis=1), coordinators_df['params'].apply(pd.Series)], axis=1)
coordinators_df_2 = coordinators_df_2.sort_values(['convocation_no'],
              ascending = [True]).reset_index().drop(['index'], axis=1)

coordinators_df_2


,convocation_no,Бытовые вопросы,Образование,Оповещение,Спорт,Праздники,Связи,Распределение стипендий,Работа с партнёрами,"Бытовые вопросы (сектор ""А"", экология)","Проблемы общежитий (сектор ""Б"", ФДС)",Спорт и праздники,Трудоустройство,Разработка Устава и Положения
0,1,Котляров Никита Владимирович,Владыкина Вероника Евгеньевна,Сапунов Кирилл Вячеславович,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Попов Леонид Андреевич,Коновалов Антон Александрович,Тропин Александр Михайлович
1,2,Котляров Никита Владимирович,Владыкина Вероника Евгеньевна,Матвеев Дмитрий Александрович,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Тен Анна Бенхиевна,Осин Руслан Владимирович,NaN
2,3,NaN,Медведева Яна Григорьевна,Матвеев Дмитрий Александрович,Антонов Сергей Валентинович,Быстрицкая Василина Васильевна,NaN,NaN,Осин Руслан Владимирович,Митькина Мария Александровна,Короткин Виталий Владимирович,NaN,NaN,NaN
3,4,Денисов Илья Олегович,Савин Павел Андреевич,Бекбулатов Рамзан Ломалиевич,Шведов Константин Анатольевич,Дорман Елена Вячеславовна,NaN,Матвеев Дмитрий Александрович,Тюрина Татьяна Игоревна,NaN,NaN,NaN,NaN,NaN
4,5,Денисов Илья Олегович,Савин Павел Андреевич,Глунчадзе Ираклий Вахтангович,Семенюк Павел Игоревич,Суркова Анастасия Юрьевна,NaN,Подкользина Евгения Борисовна,Тюрина Татьяна Игоревна,NaN,NaN,NaN,NaN,NaN
5,6,Авилова Екатерина Михайловна,Нагорных Яна Валерьевна,Саакян Вильям Рустамович,Семенюк Павел Игоревич,Муравьёва Анна Сергеевна,NaN,Глунчадзе Ираклий Вахтангович,Тонконогов Александр Владиславович,NaN,NaN,NaN,NaN,NaN
6,7,Корж Даниил Сергеевич,Воробьёва Наталья Дмитриевна,Сенотов Валерий Дмитриевич,Малахов Николай Сергеевич,Гигин Иван Сергеевич,NaN,None,Змиевский Леонид Вячеславович,NaN,NaN,NaN,NaN,NaN
7,8,Корж Даниил Сергеевич,Промыслов Платон Валерьевич,Краснова Анастасия Александровна,Малахов Николай Сергеевич,Вихорева Надежда Дмитриевна,NaN,Кухарчук Иван Андреевич,None,NaN,NaN,NaN,NaN,NaN
8,9,Фенелонов Даниил Игоревич,Саметов Максим Алексеевич,Малиновский Георгий Андреевич,None,Евдокимова Софья Александровна,NaN,Кухарчук Иван Андреевич,NaN,NaN,NaN,NaN,NaN,NaN
9,10,Дехнич Кирилл Андреевич,Кухарчук Иван Андреевич,Третьяков Пётр Максимович,Евдокимова Софья Александровна,Манахова Анастасия Николаевна,Чуб Вячеслав Валериевич,Шишков Александр Александрович,NaN,NaN,NaN,NaN,NaN,NaN


In [187]:
coordinators_df_3 = coordinators_df_2.melt(id_vars=["convocation_no"]).rename(columns={'variable':'working_group_name', 'value':'name'})
coordinators_df_3 = coordinators_df_3[coordinators_df_3['name'].notna()].reset_index().drop(['index'], axis=1)
coordinators_df_3



,convocation_no,working_group_name,name
0,1,Бытовые вопросы,Котляров Никита Владимирович
1,2,Бытовые вопросы,Котляров Никита Владимирович
2,4,Бытовые вопросы,Денисов Илья Олегович
3,5,Бытовые вопросы,Денисов Илья Олегович
4,6,Бытовые вопросы,Авилова Екатерина Михайловна
...,...,...,...
63,1,Спорт и праздники,Попов Леонид Андреевич
64,2,Спорт и праздники,Тен Анна Бенхиевна
65,1,Трудоустройство,Коновалов Антон Александрович
66,2,Трудоустройство,Осин Руслан Владимирович


In [217]:
def short_working_group_name(working_group_name):
    if 'бытовые' in str.lower(working_group_name):
        return 'Бытовые вопросы'
    elif 'общежитий' in str.lower(working_group_name):
        return 'Бытовые вопросы'
    elif 'трудоустройство' in str.lower(working_group_name):
        return 'Партнёры'
    elif 'партнёр' in str.lower(working_group_name):
        return 'Партнёры'
    elif working_group_name == 'Спорт и праздники':
        return ['Спорт', 'Праздники']
    else:
        return working_group_name
    
def is_constant(working_group_name):
    if working_group_name in ['Разработка Устава и Положения', 'Распределение стипендий']:
        return 0
    else:
        return 1

dict_working_group = coordinators_df_3.groupby(['working_group_name']).agg({'convocation_no':'min'}).reset_index().rename(columns={'convocation_no':'first_convocation_no'})
dict_working_group['short_working_group_name'] = dict_working_group['working_group_name'].apply(lambda x: short_working_group_name(x) )
dict_working_group['constant_flg'] = dict_working_group['working_group_name'].apply(lambda x: is_constant(x) )
dict_working_group = dict_working_group.sort_values(['first_convocation_no', 'constant_flg', 'working_group_name'],
              ascending = [True, False, True]).reset_index().drop(['index'], axis=1)
dict_working_group


,working_group_name,first_convocation_no,short_working_group_name,constant_flg
0,Бытовые вопросы,1,Бытовые вопросы,1
1,Образование,1,Образование,1
2,Оповещение,1,Оповещение,1
3,Спорт и праздники,1,"[Спорт, Праздники]",1
4,Трудоустройство,1,Партнёры,1
5,Разработка Устава и Положения,1,Разработка Устава и Положения,0
6,"Бытовые вопросы (сектор ""А"", экология)",3,Бытовые вопросы,1
7,Праздники,3,Праздники,1
8,"Проблемы общежитий (сектор ""Б"", ФДС)",3,Бытовые вопросы,1
9,Работа с партнёрами,3,Партнёры,1


In [226]:
dict_working_group.to_csv('ssmm_dict_working_groups.csv', sep='\t', encoding='utf-8', index=False)
dict_working_group.to_csv('../dictionaries/ssmm_dict_working_groups.csv', sep='\t', encoding='utf-8', index=False)


In [219]:
coordinators_df_3['position'] = coordinators_df_3['working_group_name'].apply(lambda x: 'Координатор РГ «{0}»'.format(x))
coordinators_df_3['position_type'] = 'Координатор'
coordinators_df_3


,convocation_no,working_group_name,name,position,position_type
0,1,Бытовые вопросы,Котляров Никита Владимирович,Координатор РГ «Бытовые вопросы»,Координатор
1,2,Бытовые вопросы,Котляров Никита Владимирович,Координатор РГ «Бытовые вопросы»,Координатор
2,4,Бытовые вопросы,Денисов Илья Олегович,Координатор РГ «Бытовые вопросы»,Координатор
3,5,Бытовые вопросы,Денисов Илья Олегович,Координатор РГ «Бытовые вопросы»,Координатор
4,6,Бытовые вопросы,Авилова Екатерина Михайловна,Координатор РГ «Бытовые вопросы»,Координатор
...,...,...,...,...,...
63,1,Спорт и праздники,Попов Леонид Андреевич,Координатор РГ «Спорт и праздники»,Координатор
64,2,Спорт и праздники,Тен Анна Бенхиевна,Координатор РГ «Спорт и праздники»,Координатор
65,1,Трудоустройство,Коновалов Антон Александрович,Координатор РГ «Трудоустройство»,Координатор
66,2,Трудоустройство,Осин Руслан Владимирович,Координатор РГ «Трудоустройство»,Координатор


In [229]:
coordinators_df_3.to_csv('ssmm_coordinators_1_11.csv', sep='\t', encoding='utf-8', index=False)
coordinators_df_3.to_csv('../library: main/ssmm_coordinators_1_11.csv', sep='\t', encoding='utf-8', index=False)


# ОСТАЛЬНОЕ

In [70]:
# position_names = sorted(list(management_df_2))
# position_names

# original list:
# 'Бытовые вопросы',
#  'Бытовые вопросы (ГЗ А, Б, Г, Е, ДСЛ, ФДС, экология)',
#  'Бытовые вопросы (сектор "А", экология)',
#  'Бытовые вопросы (сектора «А» и «Б», ФДС, экология)',
#  'Заместители представителей',
#  'Заместитель председателя',
#  'Образование',
#  'Оповещение',
#  'Праздники',
#  'Председатель',
#  'Представители в Студсовет МГУ',
#  'Представители в Студсовете МГУ',
#  'Представитель в Учёном совете',
#  'Представитель в Учёный совет мехмата',
#  'Проблемы общежитий (сектор "Б", ФДС)',
#  'Работа с партнёрами',
#  'Разработка Устава и Положения',
#  'Распределение стипендий',
#  'Связи',
#  'Секретарь',
#  'Спорт',
#  'Спорт и праздники',
#  'Трудоустройство'

In [36]:
# table_MN = pd.read_html(menu_dict['10 состав — ноябрь 2020'], match='wk_table')
# df = table_MN[0]
# df

ImportError: lxml not found, please install it

In [ ]:
# РУКОВОДСТВО

# 1 - https://vk.com/topic-33327655_26214681
# 2 - https://vk.com/wall-33327655_733
# 3 - https://vk.com/wall-33327655_1603


# 8 - https://vk.com/wall-33327655_7774
# 9 - https://vk.com/wall-33327655_8432
# 10 - https://vk.com/wall-33327655_9402
# 11 - https://vk.com/wall-33327655_10064

In [ ]:
# КООРДИНАТОРЫ

# 1 - https://vk.com/topic-33327655_26162739 - НЕ ПОЛНО, РУКАМИ
# 2 - https://vk.com/wall-33327655_733
# 3 - https://vk.com/wall-33327655_1603
# 4 - https://vk.com/wall-33327655_2354 - НЕТ ТЕКСТА
# Избраны были также и координаторы всех рабочих групп совета.
# 5 - https://vk.com/wall-33327655_3524
# 6 - https://vk.com/wall-33327655_5487
# На первом собрании Студсовета были также избраны и координаторы рабочих групп совета.
# 7 - https://vk.com/wall-33327655_6755
# 8 - https://vk.com/wall-33327655_7776
# 9 - 
# Координаторы рабочих групп
# 10 - https://vk.com/wall-33327655_9421
# 11 - https://vk.com/wall-33327655_10092